In [28]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)

Overwriting mapper.py


In [29]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0

Overwriting reducer.py


In [30]:
%%writefile -a reducer.py

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Appending to reducer.py


In [31]:
! hdfs dfs -ls -h /data/wiki/en_articles_part/articles-part

-rwxrwxrwx   1 jovyan supergroup     73.3 M 2017-11-28 21:41 /data/wiki/en_articles_part/articles-part


In [36]:
%%bash

WC_DIR="wordcount_result_"$(date +"%s%6N")
WR_DIR="wordrating_result_"$(date +"%s%6N")
NUM_REDUCERS=8

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Streaming word count" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -combiner "python reducer.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${WC_DIR} > /dev/null

# hdfs dfs -cat ${OUT_DIR}/part-00000 | head
NUM_REDUCERS=1

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Streaming word rating by count" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files reducer.py \
    -mapper "cat" \
    -reducer "sort -k2nr" \
    -input ${WC_DIR} \
    -output ${WR_DIR} > /dev/null

hdfs dfs -cat ${WR_DIR}/part-00000 | head -7 | tail -1

is	126420


18/03/22 16:53:20 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/03/22 16:53:20 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/03/22 16:53:21 INFO mapred.FileInputFormat: Total input files to process : 1
18/03/22 16:53:21 INFO mapreduce.JobSubmitter: number of splits:2
18/03/22 16:53:21 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1521700201014_0016
18/03/22 16:53:21 INFO impl.YarnClientImpl: Submitted application application_1521700201014_0016
18/03/22 16:53:21 INFO mapreduce.Job: The url to track the job: http://0175c2e98ef9:8088/proxy/application_1521700201014_0016/
18/03/22 16:53:21 INFO mapreduce.Job: Running job: job_1521700201014_0016
18/03/22 16:53:29 INFO mapreduce.Job: Job job_1521700201014_0016 running in uber mode : false
18/03/22 16:53:29 INFO mapreduce.Job:  map 0% reduce 0%
18/03/22 16:53:46 INFO mapreduce.Job:  map 19% reduce 0%
18/03/22 16:53:52 INFO mapreduce.Job:  map 29% reduce 0%
18/03/22 16:53:58 INFO 